In [1]:
import numpy as np
import numba as nb
import sys


sys.path.append("..")

from swarmalatorlib.template import Swarmalators2D

In [12]:
class MinimalPredator(Swarmalators2D):
    def __init__(self, agentsNum: int, dt: float, 
                 a: float, b: float, c: float, p: float,
                 randomSeed: int = 100) -> None:
        super().__init__(agentsNum, dt, 0, randomSeed)
        self.a = a
        self.b = b
        self.c = c
        self.p = p
        self.predatorPosition = np.array([0.0, 0.0])

    @property
    def Iatt(self) -> np.ndarray:
        """Spatial attraction: a(x_j - x_i)"""
        return self.a * self.deltaX
    
    @property
    def Irep(self) -> np.ndarray:
        """Spatial repulsion: (x_j - x_i) / |x_j - x_i|^2"""
        deltaX = self.deltaX
        return deltaX / self.distance_x_2(deltaX) ** 2

    @property
    def Fpredator(self) -> np.ndarray:
        """Effect of spatial repulsion on spatial barrier"""
        return self._Fpredator(
            position=self.positionX, 
            predatorPosition=self.predatorPosition,
            b=self.b
        )

    @staticmethod
    @nb.njit
    def _Fpredator(position: np.ndarray, predatorPosition: np.ndarray, b: float):
        delta = position - predatorPosition
        distence = delta[:, 0] ** 2 + delta[:, 1] ** 2
        return b * delta / distence.reshape(distence.shape[0], 1)

    @property
    def Fprey(self):
        return self._Fprey(
            position=self.positionX, predatorPosition=self.predatorPosition,
            c=self.c, p=self.p
        )

    @staticmethod
    @nb.njit
    def _Fprey(position: np.ndarray, predatorPosition: np.ndarray, 
               c: float, p: float):
        delta = position - predatorPosition
        distence = np.sqrt(delta[:, 0] ** 2 + delta[:, 1] ** 2) ** p
        return c / delta.shape[0] * np.sum(delta / distence.reshape(distence.shape[0], 1), axis=0)

    @staticmethod
    # @nb.njit
    def _update(
        positionX: np.ndarray, predatorPosition: np.ndarray,
        Iatt: np.ndarray, Irep: np.ndarray,
        Fpredator: np.ndarray, Fprey: np.ndarray,
        dt: float
    ):
        dim = positionX.shape[0]
        pointX = np.sum(Irep - Iatt, axis=1) / (dim - 1) + Fpredator
        positionX += pointX * dt
        predatorPosition += Fprey
        return positionX
    
    def update(self) -> None:
        self.positionX, self.predatorPosition = self._update(
            positionX=self.positionX, predatorPosition=self.predatorPosition,
            Iatt=self.Iatt, Irep=self.Irep,
            Fpredator=self.Fpredator, Fprey=self.Fprey, dt=self.dt
        )

In [13]:
model = MinimalPredator(agentsNum=1000, dt=0.05, a=1, b=2, c=1.5, p=3)

In [22]:
delta = model.positionX - model.predatorPosition
distence = np.sqrt(delta[:, 0] ** 2 + delta[:, 1] ** 2) ** 3
np.sum(delta / distence.reshape(1000, 1), axis=0) / 1000

array([149.35869664, 178.9273131 ])

In [14]:
model.Fprey

array([224.03804496, 268.39096966])